In [8]:
from sklearn.decomposition import PCA
import pickle
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import requests
import PIL.Image
from io import BytesIO
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.applications import vgg16
import tensorflow as tf
from elasticsearch import Elasticsearch
import json
from IPython.display import Image, display, Markdown, Code, HTML

In [3]:
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [9]:
es = Elasticsearch('127.0.0.1', port=9200)

In [10]:
index = 'indexproject'
source_no_vecs = ['imageId','tags', 'imgUrl']

In [6]:
settings = {
  "settings": {
    "elastiknn": True,
    "number_of_shards": 1,
    "number_of_replicas": 0
  }
}

mapping = {
  "dynamic": False,
  "properties": {
      "imageId": { "type": "keyword" },
      "featurevector": {
          "type": "elastiknn_dense_float_vector",
          "elastiknn": {
            "dims": 795,
            "model": "lsh",
            "similarity": "l2",
            "L": 60,
            "k": 3,
            "w": 2
          }
    },
    "tags": { "type": "text" },
    "imgUrl":{"type":"text","index":False}
  }
}


if not es.indices.exists(index):
    es.indices.create(index, json.dumps(settings))
    es.indices.put_mapping(json.dumps(mapping), index)
es.indices.get_mapping(index)

{'indexproject': {'mappings': {'dynamic': 'false',
   'properties': {'featurevector': {'type': 'elastiknn_dense_float_vector',
     'elastiknn': {'model': 'lsh',
      'similarity': 'l2',
      'dims': 795,
      'L': 60,
      'k': 3,
      'w': 2}},
    'imageId': {'type': 'keyword'},
    'imgUrl': {'type': 'text', 'index': False},
    'tags': {'type': 'text'}}}}}

In [11]:

with open('C:/Users/bedis/Desktop/pkl2/L.pkl', 'rb') as f:
   L = pickle.load(f)
with open('C:/Users/bedis/Desktop/pkl2/Lindice.pkl', 'rb') as f:
   Lindice= pickle.load(f)
with open('C:/Users/bedis/Desktop/pkl1/PCA795.pkl', 'rb') as f:
   PCA = pickle.load(f)

In [12]:
df=pd.read_csv('C:/Users/bedis/Desktop/flickr/cbir/photo_metadata/resultat2.csv',dtype={'id': str})

In [11]:
def image_actions():
    
    for i in Lindice:
        
        link="https://live.staticflickr.com/{x}/{y}_{z}.jpg".format(x=df.iloc[i]['flickr_server'],y=df.iloc[i]['id'],z=df.iloc[i]['flickr_secret'])
        yield {
              "_op_type": "index", "_index": index, "_id": df.iloc[i]["id"],
               "tags":df.iloc[i]['tags'],

              "imgUrl": link
            }
        

In [12]:
from elasticsearch.helpers import bulk
bulk(es, image_actions(), chunk_size=2000, max_retries=2)

c:\users\bedis\appdata\local\programs\python\python36\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


(100001, [])

In [ ]:
es.indices.refresh(index=index)
es.indices.forcemerge(index=index, max_num_segments=1, request_timeout=300)

In [12]:
model = vgg16.VGG16(weights='imagenet', include_top=True)

In [15]:
L=PCA.transform(L)
def feature_extractor(img):
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)
    
    model_extractfeatures = Model(inputs=model.input, outputs=model.get_layer('fc1').output)
    fc2_features = model_extractfeatures.predict(x)[0]
    
    
    return fc2_features/np.linalg.norm(fc2_features)

In [16]:
from elasticsearch.helpers import bulk
def vector_action():
    
    for i in range(len(Lindice)):
        
        try:
            
            

            p=L[i]
            yield { "_op_type": "update", "_index": index, "_id": df.iloc[Lindice[i]]["id"], 
                "doc": { 
                  "featurevector": { "values": p.tolist()}
                }}
        except:
            pass
        
        

In [17]:
bulk(es, vector_action(), chunk_size=1000, max_retries=2)

c:\users\bedis\appdata\local\programs\python\python36\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
Exception ignored in: <generator object vector_action at 0x0000022A971ED620>
RuntimeError: generator ignored GeneratorExit


(100001, [])

In [ ]:
es.indices.refresh(index=index)
es.indices.forcemerge(index=index, max_num_segments=1, request_timeout=1200)